In [34]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import os

In [35]:
current_path = os.getcwd()
parent_path = os.path.abspath(os.path.dirname(os.getcwd()))

In [36]:
parent_path

'd:\\YQ-SASS\\上海海关数据更新'

In [37]:
product=["机电产品",
        '农产品',
        '食品',
        '高新技术产品',
        '金属矿及矿砂',
        "高新技术产品",
        '医药材及药品',
        '计量检测分析自控仪器及器具',
        '医疗仪器及器械',
        "集成电路",
        '汽车包括底盘',
        "汽车零配件",
        '电动载人汽车',
        "自动数据处理设备及其部件",
        "自动数据处理设备及其零部件",
        "笔记本电脑",
        "手机",
        "锂离子蓄电池",
        "太阳能电池",
        "生命科学技术"]


In [38]:
import re
def findcn(name):
    res=re.findall('[\u4e00-\u9fa5]*',name)
    for pro in res:
        if len(pro)>0:
            break
        else:
            pass
    return pro

In [39]:
#将带逗号的数转化成数值，如果本身是数值，则不变
def str_to_num(x):
    try:
        return float(x.replace(',',''))
    except:
        return x  

In [40]:
def get_product_sh(parent_path):
    dir_list=os.listdir(parent_path +"\data\上海海关数据")
    df_product_sh=pd.DataFrame()
    for dirname in dir_list:
        data=pd.read_excel(parent_path +"\data\上海海关数据/"+dirname,header=3)
        data.商品编码8位=data.商品编码8位.apply(lambda x:x.strip())
        data['商品']=data.商品编码8位.apply(lambda x:findcn(x))
        data=data[data['商品'].isin(product)]
        data=data[['商品','人民币(万).1', '人民币同比(%).1']]
        data.index=range(len(data))
        data.columns=['商品','金额','同比']
        data['金额']=data['金额']/10000
        data['进口or出口']=dirname[7:9]
        data['年份']=dirname[0:4]
        df_product_sh=pd.concat([df_product_sh,data],axis=0,ignore_index=True)

    df_sh=pd.read_excel(r'../output/2.1结果.xlsx')
    df_sh['进口or出口']=df_sh['出口or进口']
    df_sh['上海金额']=df_sh['金额']
    df_sh.年份=df_sh.年份.astype('str')
    df_product_sh=df_product_sh.merge(df_sh[['年份','进口or出口','上海金额']],on=['年份','进口or出口'],how='left')
    df_product_sh['占上海比重']=df_product_sh['金额']/df_product_sh['上海金额']

    return df_product_sh
    

In [48]:
def get_product_all(parent_path):

    df_product_all=pd.DataFrame()
    dir_list=os.listdir(parent_path +"\data\全国海关数据")
    for dirname in dir_list:
        if len(dirname)<10:
            pass
        else:
            data=pd.read_excel(parent_path +"\data\全国海关数据/"+dirname,header=3)
            data.商品名称=data.商品名称.astype('str')
            data['商品']=data.商品名称.apply(lambda x:findcn(x))
            data=data[data['商品'].isin(product)]
            data=data[['商品','Unnamed: 6','Unnamed: 10']]
            data.index=range(len(data))
            data.columns=['商品','金额','同比']
            data['金额']=data['金额'].apply(lambda x:str_to_num(x))
            data['金额']=data['金额'].astype('int')
            data['金额']=data['金额']/10000
            data['进口or出口']=dirname[6:8]
            data['年份']=dirname[0:4]
            print(data)
            df_product_all=pd.concat([df_product_all,data],axis=0,ignore_index=True)
            df_product_all.rename(columns={'金额':'全国金额','同比':'全国同比'},inplace=True)
    return df_product_all

In [49]:
df_product_sh = get_product_sh(parent_path)
df_product_all = get_product_all(parent_path)

def get_res_product(df_product_sh,df_product_all):
    res=df_product_sh.merge(df_product_all,on=['商品','进口or出口','年份'],how='left')
    res['上海占全国比重']=res['金额']/res['全国金额']
    res.drop_duplicates(inplace=True)
    res.rename(columns={'金额':"上海进出口该商品金额",'同比':'上海同比','上海金额':"上海进出口商品总额",'全国金额':"全国进出口该商品金额"},inplace=True)
    res['年份']=res['年份'].astype('int')
    res['商品'].replace('自动数据处理设备及其部件','自动数据处理设备及其零部件',inplace=True)
    res['占上海比重']=res['占上海比重']*100
    res['上海占全国比重']=res['上海占全国比重']*100
    return res 

             商品           金额     同比 进口or出口    年份
0  自动数据处理设备及其部件   11415.0942    0.5     出口  2019
1         太阳能电池    1319.4282   47.5     出口  2019
2          集成电路    7008.2496   25.3     出口  2019
3        电动载人汽车      95.0255  178.9     出口  2019
4         汽车零配件    3654.8088    0.8     出口  2019
5       医疗仪器及器械     890.9592   18.4     出口  2019
6           农产品    5420.4043    3.5     出口  2019
7          机电产品  100631.2490    4.4     出口  2019
8        高新技术产品   50426.6590    2.1     出口  2019
9        生命科学技术    2301.7182    6.6     出口  2019
               商品          金额     同比 进口or出口    年份
0              食品     25.4352     11     进口  2019
1    自动数据处理设备及其部件   2238.2175    3.2     进口  2019
2            集成电路  21079.4664    2.4     进口  2019
3          电动载人汽车    461.5371  115.8     进口  2019
4           汽车零配件   2111.8946   -8.6     进口  2019
5         医疗仪器及器械    858.1624   15.6     进口  2019
6   计量检测分析自控仪器及器具   2577.6612    1.9     进口  2019
7             农产品  10332.3779   14.7     进口  2019
8          

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [55]:
dir_list

['2019.xls',
 '2019全国出口数据.xlsx',
 '2019全国进口数据.xlsx',
 '2020.xls',
 '2020全国出口数据.xls',
 '2020全国进口数据.xls',
 '2021.xls',
 '2021全国出口数据.xls',
 '2021全国进口数据.xls',
 '2022.xls',
 '2022全国出口数据.xls',
 '2022全国进口数据.xls',
 '2023.xls',
 '2023全国出口数据.xls',
 '2023全国进口数据.xls',
 '2024.xls',
 '2024全国出口数据.xls',
 '2024全国进口数据.xls']

In [57]:
df_all=pd.DataFrame()
dir_list=os.listdir(parent_path +"\data\全国海关数据")

for dirname in dir_list:
    if len(dirname)<10:
        pass
    else:
        data=pd.read_excel(r'D:\YQ-SASS\数字贸易\上海进出口数据\上海进出口数据v6\上海进出口数据\全国海关数据/'+dirname,header=3)
        print(dirname)  
        data.商品名称=data.商品名称.astype('str')
        data['商品']=data.商品名称.apply(lambda x:findcn(x))
        data=data[data['商品'].isin(product)]
        data=data[['商品','Unnamed: 6','Unnamed: 10']]
        data.index=range(len(data))
        data.columns=['商品','金额','同比']
        data['金额']=data['金额'].apply(lambda x:str_to_num(x))
        data['金额']=data['金额'].astype('int')
        data['金额']=data['金额']/10000
        data['进口or出口']=dirname[6:8]
        data['年份']=dirname[0:4]
        df_all=pd.concat([df_all,data],axis=0,ignore_index=True)

2019全国出口数据.xlsx
2019全国进口数据.xlsx
2020全国出口数据.xls
2020全国进口数据.xls
2021全国出口数据.xls
2021全国进口数据.xls
2022全国出口数据.xls
2022全国进口数据.xls
2023全国出口数据.xls
2023全国进口数据.xls
2024全国出口数据.xls
2024全国进口数据.xls


In [60]:
df_product_all=pd.DataFrame()
dir_list=os.listdir(parent_path +"\data\全国海关数据")
for dirname in dir_list:
    if len(dirname)<10:
        pass
    else:
        print(dirname)
        data=pd.read_excel(parent_path +"\data\全国海关数据/"+dirname,header=3)
        data.商品名称=data.商品名称.astype('str')
        data['商品']=data.商品名称.apply(lambda x:findcn(x))
        data=data[data['商品'].isin(product)]
        data=data[['商品','Unnamed: 6','Unnamed: 10']]
        data.index=range(len(data))
        data.columns=['商品','金额','同比']
        data['金额']=data['金额'].apply(lambda x:str_to_num(x))
        data['金额']=data['金额'].astype('int')
        data['金额']=data['金额']/10000
        data['进口or出口']=dirname[6:8]
        data['年份']=dirname[0:4]
        print(data)
        df_product_all=pd.concat([df_product_all,data],axis=0,ignore_index=True)
df_product_all.rename(columns={'金额':'全国金额','同比':'全国同比'},inplace=True)

2019全国出口数据.xlsx
             商品           金额     同比 进口or出口    年份
0  自动数据处理设备及其部件   11415.0942    0.5     出口  2019
1         太阳能电池    1319.4282   47.5     出口  2019
2          集成电路    7008.2496   25.3     出口  2019
3        电动载人汽车      95.0255  178.9     出口  2019
4         汽车零配件    3654.8088    0.8     出口  2019
5       医疗仪器及器械     890.9592   18.4     出口  2019
6           农产品    5420.4043    3.5     出口  2019
7          机电产品  100631.2490    4.4     出口  2019
8        高新技术产品   50426.6590    2.1     出口  2019
9        生命科学技术    2301.7182    6.6     出口  2019
2019全国进口数据.xlsx
               商品          金额     同比 进口or出口    年份
0              食品     25.4352     11     进口  2019
1    自动数据处理设备及其部件   2238.2175    3.2     进口  2019
2            集成电路  21079.4664    2.4     进口  2019
3          电动载人汽车    461.5371  115.8     进口  2019
4           汽车零配件   2111.8946   -8.6     进口  2019
5         医疗仪器及器械    858.1624   15.6     进口  2019
6   计量检测分析自控仪器及器具   2577.6612    1.9     进口  2019
7             农产品  10332.3779

In [54]:
df_product_all

,商品,全国金额,全国同比,进口or出口,年份,全国金额,全国同比
0,自动数据处理设备及其部件,11415.0942,0.5,出口,2019,NaN,NaN
1,太阳能电池,1319.4282,47.5,出口,2019,NaN,NaN
2,集成电路,7008.2496,25.3,出口,2019,NaN,NaN
3,电动载人汽车,95.0255,178.9,出口,2019,NaN,NaN
4,汽车零配件,3654.8088,0.8,出口,2019,NaN,NaN
5,医疗仪器及器械,890.9592,18.4,出口,2019,NaN,NaN
6,农产品,5420.4043,3.5,出口,2019,NaN,NaN
7,机电产品,100631.2490,4.4,出口,2019,NaN,NaN
8,高新技术产品,50426.6590,2.1,出口,2019,NaN,NaN
9,生命科学技术,2301.7182,6.6,出口,2019,NaN,NaN


In [53]:
data

,商品,金额,同比,进口or出口,年份
0,农产品,5266.4671,-2.8,出口,2020
1,医药材及药品,1593.9150,27.3,出口,2020
2,机电产品,106607.7265,6,出口,2020
3,自动数据处理设备及其零部件,14598.6124,12,出口,2020
4,笔记本电脑,6124.9668,20.4,出口,2020
5,锂离子蓄电池,1102.5505,22.6,出口,2020
6,手机,8646.6756,0.4,出口,2020
7,太阳能电池,1370.0488,3.9,出口,2020
8,太阳能电池,1370.0488,3.9,出口,2020
9,集成电路,8056.2610,15,出口,2020


In [ ]:
res.to_excel('表2-2结果.xlsx',index=False)